In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.tsa.holtwinters import SimpleExpSmoothing   
from statsmodels.tsa.holtwinters import ExponentialSmoothing

## Load data and decompose

In [ ]:
file = 'airline-passengers.csv'
airline = pd.read_csv(file, index_col='Month', parse_dates=True)

# Set the frequency of the date time index as Monthly start as indicated by the data
airline.index.freq = 'MS'
airline.shape

In [ ]:
airline.head()

In [ ]:
airline.describe()

In [ ]:
airline.plot(title='Passengers Data');
#airline['Passengers'].plot(title='Passengers Data', legend=True)

In [ ]:
decomp = seasonal_decompose(airline['Passengers'], model='multiplicative')
decomp.plot();

In [ ]:
airline_train = airline[['Passengers']][:120]
airline_test = airline[['Passengers']][120:]

## Simple Exponential Smoothing

In [ ]:
# Set the value of Alpha and define m (Time Period)
m = 12
alpha = 1/(2*m)
two_years = m*2

In [ ]:
model1 = SimpleExpSmoothing(airline_train['Passengers']).fit(smoothing_level=alpha, optimized=False, use_brute=True)
airline_train['HWES1'] = model1.fittedvalues
airline_test['HWES1'] = model1.forecast(two_years)

In [ ]:
plt.title('Holt Winters Single Exponential Smoothing')
plt.plot(airline_train['Passengers'], label='train')
plt.plot(airline_test['Passengers'], label='test')
plt.plot(airline_train['HWES1'], color='C2', label='HWSE1')
plt.plot(airline_test['HWES1'], color='C2')
plt.legend()

## Double Exponential Smoothing

In [ ]:
model2_add = ExponentialSmoothing(airline_train['Passengers'], trend='add').fit()
airline_train['HWES2_ADD'] = model2_add.fittedvalues
airline_test['HWES2_ADD'] = model2_add.forecast(two_years)

model2_mul = ExponentialSmoothing(airline_train['Passengers'], trend='mul').fit()
airline_train['HWES2_MUL'] = model2_mul.fittedvalues
airline_test['HWES2_MUL'] = model2_mul.forecast(two_years)

In [ ]:
plt.title('Holt Winters Double Exponential Smoothing: Additive and Multiplicative Trend')
plt.plot(airline_train['Passengers'], label='train')
plt.plot(airline_test['Passengers'], label='test')
plt.plot(airline_train['HWES2_ADD'], color='C3', label='HWES2_ADD')
plt.plot(airline_test['HWES2_ADD'], color='C3')
plt.plot(airline_train['HWES2_MUL'], color='C4', label='HWES2_MUL')
plt.plot(airline_test['HWES2_MUL'], color='C4')
plt.legend()

In [ ]:
shift = -1
plt.plot(airline_train['Passengers'])
plt.plot(airline_train['HWES2_ADD'].shift(shift))
plt.plot(airline_train['HWES2_MUL'].shift(shift))

## Triple Exponential Smoothing

In [ ]:
model3_add = ExponentialSmoothing(airline_train['Passengers'], trend='add', seasonal='add', seasonal_periods=12).fit(method='ls')
airline_train['HWES3_ADD'] = model3_add.fittedvalues
airline_test['HWES3_ADD'] = model3_add.forecast(two_years)

model3_mul = ExponentialSmoothing(airline_train['Passengers'], trend='mul', seasonal='mul', seasonal_periods=12).fit(method='ls')
airline_train['HWES3_MUL'] = model3_mul.fittedvalues
airline_test['HWES3_MUL'] = model3_mul.forecast(two_years)

In [ ]:
plt.title('Holt Winters Triple Exponential Smoothing: Additive and Multiplicative Trend')
plt.plot(airline_train['Passengers'], label='train')
plt.plot(airline_test['Passengers'], label='test')
plt.plot(airline_train['HWES3_ADD'], color='C5', label='HWES3_ADD')
plt.plot(airline_test['HWES3_ADD'], color='C5')
plt.plot(airline_train['HWES3_MUL'], color='C6', label='HWES3_ADD')
plt.plot(airline_test['HWES3_MUL'], color='C6')
plt.legend()

## Compare models

In [ ]:
alpha = 0.5
plt.title('Holt Winters Exponential Smoothing')
plt.plot(airline_train['Passengers'], alpha=alpha, label='train')
plt.plot(airline_test['Passengers'], alpha=alpha, label='test')
plt.plot(airline_train['HWES1'], color='C2', alpha=alpha, label='HWES1')
plt.plot(airline_test['HWES1'], color='C2', alpha=alpha)
plt.plot(airline_train['HWES2_ADD'], color='C3', alpha=alpha, label='HWES2_ADD')
plt.plot(airline_test['HWES2_ADD'], color='C3', alpha=alpha)
plt.plot(airline_train['HWES2_MUL'], color='C4', alpha=alpha, label='HWES2_MUL')
plt.plot(airline_test['HWES2_MUL'], color='C4', alpha=alpha)
plt.plot(airline_train['HWES3_ADD'], color='C5', alpha=alpha, label='HWES3_ADD')
plt.plot(airline_test['HWES3_ADD'], color='C5', alpha=alpha)
plt.plot(airline_train['HWES3_MUL'], color='C6', alpha=alpha, label='HWES3_MUL')
plt.plot(airline_test['HWES3_MUL'], color='C6', alpha=alpha)
#plt.xlim(airline.index[108], airline.index[143])
plt.legend()
plt.tight_layout()

In [ ]:
model1.resid.plot(kind='kde', color='C2', label='HWES1')
model2_add.resid.plot(kind='kde', color='C3', label='HWES2_ADD')
model2_mul.resid.plot(kind='kde', color='C4', label='HWES2_MUL')
model3_add.resid.plot(kind='kde', color='C5', label='HWES3_ADD')
model3_mul.resid.plot(kind='kde', color='C6', label='HWES3_MUL')
plt.legend()

In [ ]:
two_decades = 240
plt.plot(airline_train['Passengers'], label='train')
plt.plot(airline_test['Passengers'], label='test')
plt.plot(model1.forecast(two_decades), label='HWES1')
plt.plot(model2_add.forecast(two_decades), label='HWES2_ADD')
plt.plot(model2_mul.forecast(two_decades), label='HWES2_MUL')
plt.plot(model3_add.forecast(two_decades), label='HWES3_ADD')
plt.plot(model3_mul.forecast(two_decades), label='HWES3_MUL')
plt.legend()

In [ ]:
def calculate_metrics(models):
    aic = np.array([model.aic for model in models]).reshape(len(models),1)
    cols = airline_train.columns
    
    mse = []
    mae = []
    for col in cols[1:]:
        train_mse = np.mean((airline_train['Passengers'] - airline_train[col]) ** 2)
        test_mse = np.mean((airline_test['Passengers'] - airline_test[col]) ** 2)
        mse.append([train_mse, test_mse])
    
        train_mae = np.mean(np.abs(airline_train['Passengers'] - airline_train[col]))
        test_mae = np.mean(np.abs(airline_test['Passengers'] - airline_test[col]))
        mae.append([train_mae, test_mae])
    mse = np.array(mse)
    mae = np.array(mae)
    metrics = pd.DataFrame(np.hstack((aic, mse, mae)).T, columns=cols[1:],
                           index=['AIC', 'Train MSE', 'Test MSE', 'Train MAE', 'Test MAE'])

    return metrics

In [ ]:
models = [model1, model2_add, model2_mul, model3_add, model3_mul]

In [ ]:
metrics = calculate_metrics(models)

In [ ]:
metrics

In [ ]:
model3_mul.summary()